In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [68]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
train.get_dtype_counts()

float64     3
int64      35
object     43
dtype: int64

In [4]:
test.get_dtype_counts()

float64    11
int64      26
object     43
dtype: int64

In [69]:
full = pd.concat([train,test],ignore_index=True)

d:\python\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [6]:
full.drop(['Id','SalePrice'],axis=1,inplace=True)

In [7]:
tmp = full.isnull().sum()
tmp[tmp>0]

Alley           2721
BsmtCond          82
BsmtExposure      82
BsmtFinSF1         1
BsmtFinSF2         1
BsmtFinType1      79
BsmtFinType2      80
BsmtFullBath       2
BsmtHalfBath       2
BsmtQual          81
BsmtUnfSF          1
Electrical         1
Exterior1st        1
Exterior2nd        1
Fence           2348
FireplaceQu     1420
Functional         2
GarageArea         1
GarageCars         1
GarageCond       159
GarageFinish     159
GarageQual       159
GarageType       157
GarageYrBlt      159
KitchenQual        1
LotFrontage      486
MSZoning           4
MasVnrArea        23
MasVnrType        24
MiscFeature     2814
PoolQC          2909
SaleType           1
TotalBsmtSF        1
Utilities          2
dtype: int64

In [8]:
cols1 = ["PoolQC" , "MiscFeature", "Alley", "Fence", "FireplaceQu", "GarageQual", "GarageCond", "GarageFinish", "GarageYrBlt", "GarageType", "BsmtExposure", "BsmtCond", "BsmtQual", "BsmtFinType2", "BsmtFinType1", "MasVnrType"]
for col in cols1:
    full[col].fillna("None",inplace=True)

In [9]:
cols=["MasVnrArea", "BsmtUnfSF", "TotalBsmtSF", "GarageCars", "BsmtFinSF2", "BsmtFinSF1", "GarageArea"]
for col in cols:
    full[col].fillna(0, inplace=True)

In [10]:
cols2 = ["MSZoning", "BsmtFullBath", "BsmtHalfBath", "Utilities", "Functional", "Electrical", "KitchenQual", "SaleType","Exterior1st", "Exterior2nd"]
for col in cols2:
    full[col].fillna(full[col].mode()[0], inplace=True)

In [11]:
full["LotFrontage"].fillna(np.sqrt(full["LotArea"]),inplace=True)

In [12]:
for col in cols2:
    full[col]=full[col].astype(str)##astype来进行数据转换成字符串类型

In [15]:
from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()

In [16]:
full['GarageYrBlt'] = full['GarageYrBlt'].astype(str)
full['GarageYrBlt'] = lab.fit_transform(full.GarageYrBlt)

In [17]:
full = pd.get_dummies(full)

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler,StandardScaler

In [20]:
model = LinearRegression()
n_train = train.shape[0]#行数

#划分数据集
X = full[:n_train]##相当于取出X
y = train.SalePrice#取出训练集的标签
test_x = full[n_train:]#1461行取到最后

In [21]:
model.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [22]:
tmp = test_x.isnull().sum()
tmp[tmp>0]

Series([], dtype: int64)

In [23]:
saleprice = model.predict(test_x)

In [24]:
result=pd.DataFrame({'Id':test.Id, 'SalePrice':saleprice})
result.to_csv("submission1.csv",index=False)

In [25]:
import xgboost as xgb
xg_reg=xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.4,learning_rate=0.01,max_depth=8,alpha=10,n_estimators=600,subsample=0.7)

In [26]:
xg_reg.fit(X,y)

d:\python\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
d:\python\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0, importance_type='gain',
       learning_rate=0.01, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.7)

In [27]:
pred=xg_reg.predict(test_x)  
result2=pd.DataFrame({'Id':test.Id, 'SalePrice':pred})
result2.to_csv("submission1.csv",index=False)

# 降维

In [38]:
X_scaled = StandardScaler().fit_transform(X)
y_log = np.log(y)

d:\python\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype uint8, int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
d:\python\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [42]:
X_scaled

array([[-0.79343379,  1.16185159, -0.11633929, ...,  0.06423821,
         0.02618016, -0.02618016],
       [ 0.25714043, -0.79516323, -0.11633929, ...,  0.06423821,
         0.02618016, -0.02618016],
       [-0.62782603,  1.18935062, -0.11633929, ...,  0.06423821,
         0.02618016, -0.02618016],
       ...,
       [ 0.06565646,  1.84474434, -0.11633929, ...,  0.06423821,
         0.02618016, -0.02618016],
       [-0.21898188, -0.79516323, -0.11633929, ...,  0.06423821,
         0.02618016, -0.02618016],
       [ 0.2416147 , -0.79516323, -0.11633929, ...,  0.06423821,
         0.02618016, -0.02618016]])

In [48]:
from sklearn.linear_model import Lasso##运用算法来进行训练集的得到特征的重要性，特征选择的一个作用是，wrapper基础模型
lasso=Lasso(alpha=0.001)
lasso.fit(X_scaled,y_log)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [50]:
FI_lasso = pd.DataFrame({"Feature Importance":lasso.coef_}, index=X.columns)#索引和重要性做成dataframe形式

In [51]:
FI_lasso.sort_values('Feature Importance', ascending=False)

,Feature Importance
GrLivArea,0.120560
OverallQual,0.062502
YearBuilt,0.046842
OverallCond,0.039235
TotalBsmtSF,0.037424
BsmtFinSF1,0.025276
Neighborhood_Crawfor,0.022442
GarageArea,0.021862
LotArea,0.018918
GarageCars,0.018218


# 原始特征降维

In [58]:
tmp = train.corr()['SalePrice']

In [72]:
delete_index =tmp[tmp<0.2].index.tolist()

In [75]:
delete_index.remove('Id')

In [76]:
delete_index

['MSSubClass',
 'OverallCond',
 'BsmtFinSF2',
 'LowQualFinSF',
 'BsmtHalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [70]:
full.shape

(2919, 81)

In [71]:
full.drop(['Id','SalePrice'],axis=1,inplace=True)

In [77]:
full.drop(delete_index,axis=1,inplace=True)

In [78]:
full.shape

(2919, 65)

In [79]:
cols1 = ["PoolQC" , "MiscFeature", "Alley", "Fence", "FireplaceQu", "GarageQual", "GarageCond", "GarageFinish", "GarageYrBlt", "GarageType", "BsmtExposure", "BsmtCond", "BsmtQual", "BsmtFinType2", "BsmtFinType1", "MasVnrType"]
for col in cols1:
    full[col].fillna("None",inplace=True)

In [82]:
cols=["MasVnrArea", "BsmtUnfSF", "TotalBsmtSF", "GarageCars", "BsmtFinSF1", "GarageArea"]
for col in cols:
    full[col].fillna(0, inplace=True)

In [84]:
cols2 = ["MSZoning", "BsmtFullBath", "Utilities", "Functional", "Electrical", "KitchenQual", "SaleType","Exterior1st", "Exterior2nd"]
for col in cols2:
    full[col].fillna(full[col].mode()[0], inplace=True)

In [85]:
full["LotFrontage"].fillna(np.sqrt(full["LotArea"]),inplace=True)

In [89]:
full['GarageYrBlt'] = full['GarageYrBlt'].astype(str)
full['GarageYrBlt'] = lab.fit_transform(full.GarageYrBlt)

In [90]:
full = pd.get_dummies(full)

In [92]:
n_train = train.shape[0]#行数

#划分数据集
X = full[:n_train]##相当于取出X
y = train.SalePrice#取出训练集的标签
test_x = full[n_train:]#1461行取到最后

In [93]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler,StandardScaler

model = LinearRegression()

In [94]:
model.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [95]:
saleprice = model.predict(test_x)

In [96]:
result=pd.DataFrame({'Id':test.Id, 'SalePrice':saleprice})
result.to_csv("submission1.csv",index=False)

In [97]:
xg_reg.fit(X,y)

d:\python\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
d:\python\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0, importance_type='gain',
       learning_rate=0.01, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.7)

In [98]:
pred=xg_reg.predict(test_x)  
result2=pd.DataFrame({'Id':test.Id, 'SalePrice':pred})
result2.to_csv("submission2.csv",index=False)